In [1]:
import numpy as np
import json
import os

def generate_nodes(N, space_size=400, filename="nodes.json", initial_energy=100.0):
    """
    Sinh N node cảm biến ngẫu nhiên trong không gian 3D với residual energy
    
    N: số lượng node
    space_size: kích thước không gian
    filename: tên file lưu
    initial_energy: năng lượng ban đầu (E0)
    """
    
    np.random.seed(0)  # để kết quả lặp lại
    node_positions = np.random.rand(N, 3) * space_size
    
    # Sinh residual energy ngẫu nhiên (50-100% của initial energy)
    np.random.seed(42)  # Seed khác để energy không phụ thuộc vào vị trí
    residual_energies = 100

    data = []
    for i in range(N):
        data.append({
            "id": i,
            "x": float(node_positions[i][0]),
            "y": float(node_positions[i][1]),
            "z": float(node_positions[i][2]),
            "residual_energy": float(residual_energies[i]),
            "initial_energy": float(initial_energy)
        })

    
    os.makedirs("input_data", exist_ok=True)
    filepath = f"input_data/{filename}"
    with open(filepath, "w") as f:
        json.dump(data, f, indent=4)

    print(f"Đã tạo file {filepath} chứa {N} node với residual energy.")
    print(f"Energy range: {residual_energies.min():.1f} - {residual_energies.max():.1f}")

# Sinh dữ liệu với energy information
for N in [20, 100, 200, 500, 1000]:
    generate_nodes(N, space_size=100, filename=f"nodes_{N}.json", initial_energy=100.0)

TypeError: 'int' object is not subscriptable

In [6]:
import numpy as np

def calculate_number_clusters(nodes, base_station=(0, 0, 0), space_size=400):
    """
    Tính số cụm optimal theo công thức từ bài báo:
    K = √( / πd_tobs)
    
    nodes: tọa độ 3D của các node
    base_station: tọa độ base station (mặc định tại gốc tọa độ)
    """
    N = len(nodes)
    base_pos = np.array(base_station)
    
    # Tính khoảng cách trung bình từ các node đến base station
    distances = np.linalg.norm(nodes - base_pos, axis=1)
    d_tobs = np.mean(distances)
    
    # Áp dụng công thức K = √(N*L / πd_tobs)
    K_optimal = np.sqrt(N * space_size / (np.pi * d_tobs))
    K_optimal = max(1, int(np.round(K_optimal)))  # Đảm bảo K >= 1 và là số nguyên
    
    print(f"N = {N}, d_tobs = {d_tobs:.2f}, K_optimal = {K_optimal}")
    return K_optimal

In [ ]:
import numpy as np
from sklearn.cluster import KMeans

def cluster_split(nodes, node_ids, node_data=None, r_sen=50, R=20, max_depth=10, depth=0):
    """
    Hàm phân cụm lặp theo Algorithm 1
    nodes: tọa độ 3D của các node
    node_ids: list id tương ứng của các node
    node_data: dictionary chứa thông tin đầy đủ của nodes (bao gồm energy)
    r_sen: bán kính truyền tải tối đa của node, giả sử là 100m
    R: số lượng node tối đa trong 1 cụm, cho là 20
    max_depth: độ sâu đệ quy tối đa
    """
    
    center = np.mean(nodes, axis=0)
    dists = np.linalg.norm(nodes - center, axis=1) # khoảng cách từ tâm đến các node
    if (len(nodes) <= R and np.all(dists <= r_sen)) or depth >= max_depth:
        return [{
            "node_ids": node_ids,
            "nodes": nodes,
            "center": center,
            "node_data": node_data if node_data else {}
        }]

    # Kmeans với k=2 để chia cụm
    kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
    labels = kmeans.fit_predict(nodes)

    clusters = []
    for i in range(2):
        sub_nodes = nodes[labels == i]
        sub_ids = [node_ids[j] for j in range(len(node_ids)) if labels[j] == i]
        
        # Tạo sub_node_data cho cluster con
        sub_node_data = {}
        if node_data:
            for node_id in sub_ids:
                if node_id in node_data:
                    sub_node_data[node_id] = node_data[node_id]
        
        clusters += cluster_split(sub_nodes, sub_ids, sub_node_data, r_sen, R, max_depth, depth + 1)

    return clusters


def choose_cluster_head(cluster, node_data_dict):
    """
    Chọn cluster head theo công thức desirableness factor:
    Q = d_tocenter^(E0/E_current)
    
    Node có Q MAX sẽ được chọn làm cluster head
    
    cluster: thông tin cluster
    node_data_dict: dictionary chứa thông tin đầy đủ của tất cả nodes
    """
    nodes = cluster["nodes"]
    center = cluster["center"]
    node_ids = cluster["node_ids"]

    # Tính khoảng cách từ mỗi node đến tâm cụm
    dists_to_center = np.linalg.norm(nodes - center, axis=1)
    
    max_Q = -1
    best_cluster_head = node_ids[0]  # Default fallback
    
    print(f"\n=== Chọn Cluster Head cho cụm có {len(node_ids)} nodes ===")
    
    for i, node_id in enumerate(node_ids):
        if node_id in node_data_dict:
            node_info = node_data_dict[node_id]
            
            # Lấy thông tin energy
            E_current = node_info.get('residual_energy', 100.0)
            E0 = node_info.get('initial_energy', 100.0)
            
            # Tính desirableness factor: Q = d_tocenter^(E0/E_current)
            d_tocenter = dists_to_center[i]
            
            # Tránh chia cho 0 và overflow
            if E_current <= 0:
                E_current = 0.1  # Năng lượng rất thấp
            
            energy_ratio = E0 / E_current
            
            # Tính Q - node có Q MAX sẽ được chọn
            Q = d_tocenter ** energy_ratio
            
            print(f"  Node {node_id}: d={d_tocenter:.2f}, E_cur={E_current:.1f}, "
                  f"E0={E0:.1f}, ratio={energy_ratio:.2f}, Q={Q:.3f}")
            
            # Chọn node có Q lớn nhất
            if Q > max_Q:
                max_Q = Q
                best_cluster_head = node_id
        else:
            # Fallback: chọn node gần tâm nhất nếu không có thông tin energy
            print(f"  Node {node_id}: No energy data, using distance only")
            if i == 0 or dists_to_center[i] < dists_to_center[node_ids.index(best_cluster_head)]:
                best_cluster_head = node_id
    
    print(f"Chọn Node {best_cluster_head} làm Cluster Head (Q_max = {max_Q:.3f})")
    return best_cluster_head

In [16]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

input_folder = "D:\\Year 4\\tiến hóa\\project\\UWSN_greedy\\input_data_even"
output_folder = "D:\\Year 4\\tiến hóa\\project\\UWSN_greedy\\output_data_kmeans"
os.makedirs(output_folder, exist_ok=True)
draw_folder = "D:\\Year 4\\tiến hóa\\project\\UWSN_greedy\\draw_output_kmeans"
os.makedirs(draw_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.startswith("nodes_") and filename.endswith(".json"):
        # Lấy số lượng node từ tên file
        number_nodes = filename.split("_")[1].split(".")[0]
        # Đọc dữ liệu từ file input
        with open(os.path.join(input_folder, filename), "r") as f:
            data = json.load(f)
        node_positions = np.array([[d["x"], d["y"], d["z"]] for d in data])
        node_ids = [d["id"] for d in data]
        # Phân cụm
        k = calculate_number_clusters(node_positions, base_station=(0, 0, 0))
        print(f"Số cụm: {k}")
        clusters_raw = cluster_split(node_positions, node_ids, R=int(number_nodes)//k)
        # Tạo output
        clusters_output = {}
        for i, c in enumerate(clusters_raw):
            ch = choose_cluster_head(c, c["node_data"])
            clusters_output[i] = {
                "nodes": c["node_ids"],
                "center": tuple(np.round(c["center"], 2)),
                "cluster_head": int(ch)
            }
        # Xuất ra file
        out_path = os.path.join(output_folder, f"nodes_{number_nodes}.json")
        with open(out_path, "w") as f:
            json.dump(clusters_output, f, indent=4)
        print(f"Đã xuất file {out_path}")
        # Vẽ và lưu hình
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')
        colors = plt.cm.get_cmap('tab10', len(clusters_output))
        for cid, info in clusters_output.items():
            nodes = np.array([node_positions[nid] for nid in info['nodes']])
            ax.scatter(nodes[:, 0], nodes[:, 1], nodes[:, 2],
                        label=f'Cụm {cid}',
                        color=colors(cid))
            ch_pos = node_positions[info['cluster_head']]
            ax.scatter(ch_pos[0], ch_pos[1], ch_pos[2],
                        color=colors(cid),
                        marker='*', s=80, edgecolor='k')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.legend()
        plt.title(f'Phân cụm các node cảm biến ({number_nodes} node)')
        ax.view_init(elev=30, azim=30)
        plt.tight_layout()
        draw_path = os.path.join(draw_folder, f"nodes_{number_nodes}.png")
        plt.savefig(draw_path)
        plt.close(fig)
        print(f"Đã lưu hình vẽ {draw_path}")

N = 100, d_tobs = 360.48, K_optimal = 6
Số cụm: 6

=== Chọn Cluster Head cho cụm có 6 nodes ===
  Node 10: No energy data, using distance only
  Node 15: No energy data, using distance only
  Node 16: No energy data, using distance only
  Node 17: No energy data, using distance only
  Node 20: No energy data, using distance only
  Node 21: No energy data, using distance only
Chọn Node 16 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 2 nodes ===
  Node 45: No energy data, using distance only
  Node 46: No energy data, using distance only
Chọn Node 45 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 8 nodes ===
  Node 35: No energy data, using distance only
  Node 36: No energy data, using distance only
  Node 37: No energy data, using distance only
  Node 40: No energy data, using distance only
  Node 41: No energy data, using distance only
  Node 60: No energy data, using distance only
  Node 61: No energy data, using distance only
  Node 66: No

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10640\2894817846.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', len(clusters_output))


Đã lưu hình vẽ D:\Year 4\tiến hóa\project\UWSN_greedy\draw_output_kmeans\nodes_100.png
N = 1000, d_tobs = 384.50, K_optimal = 18
Số cụm: 18

=== Chọn Cluster Head cho cụm có 16 nodes ===
  Node 0: No energy data, using distance only
  Node 1: No energy data, using distance only
  Node 10: No energy data, using distance only
  Node 11: No energy data, using distance only
  Node 100: No energy data, using distance only
  Node 101: No energy data, using distance only
  Node 102: No energy data, using distance only
  Node 110: No energy data, using distance only
  Node 111: No energy data, using distance only
  Node 112: No energy data, using distance only
  Node 200: No energy data, using distance only
  Node 201: No energy data, using distance only
  Node 202: No energy data, using distance only
  Node 210: No energy data, using distance only
  Node 211: No energy data, using distance only
  Node 212: No energy data, using distance only
Chọn Node 101 làm Cluster Head (Q_max = -1.000)

==

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10640\2894817846.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', len(clusters_output))


Đã lưu hình vẽ D:\Year 4\tiến hóa\project\UWSN_greedy\draw_output_kmeans\nodes_1000.png
N = 20, d_tobs = 354.69, K_optimal = 3
Số cụm: 3

=== Chọn Cluster Head cho cụm có 2 nodes ===
  Node 10: No energy data, using distance only
  Node 13: No energy data, using distance only
Chọn Node 13 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 1 nodes ===
  Node 0: No energy data, using distance only
Chọn Node 0 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 3 nodes ===
  Node 9: No energy data, using distance only
  Node 18: No energy data, using distance only
  Node 19: No energy data, using distance only
Chọn Node 18 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 2 nodes ===
  Node 2: No energy data, using distance only
  Node 5: No energy data, using distance only
Chọn Node 2 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 1 nodes ===
  Node 1: No energy data, using distance only
Chọn Node 1 làm Cluster Head 

...

Đã lưu hình vẽ D:\Year 4\tiến hóa\project\UWSN_greedy\draw_output_kmeans\nodes_20.png
N = 200, d_tobs = 370.75, K_optimal = 8
Số cụm: 8

=== Chọn Cluster Head cho cụm có 5 nodes ===
  Node 150: No energy data, using distance only
  Node 151: No energy data, using distance only
  Node 156: No energy data, using distance only
  Node 186: No energy data, using distance only
  Node 192: No energy data, using distance only
Chọn Node 156 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 7 nodes ===
  Node 121: No energy data, using distance only
  Node 157: No energy data, using distance only
  Node 162: No energy data, using distance only
  Node 163: No energy data, using distance only
  Node 164: No energy data, using distance only
  Node 198: No energy data, using distance only
  Node 199: No energy data, using distance only
Chọn Node 163 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 8 nodes ===
  Node 116: No energy data, using distance only
  Node

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10640\2894817846.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', len(clusters_output))


Đã lưu hình vẽ D:\Year 4\tiến hóa\project\UWSN_greedy\draw_output_kmeans\nodes_200.png
N = 500, d_tobs = 380.04, K_optimal = 13
Số cụm: 13

=== Chọn Cluster Head cho cụm có 7 nodes ===
  Node 1: No energy data, using distance only
  Node 2: No energy data, using distance only
  Node 3: No energy data, using distance only
  Node 11: No energy data, using distance only
  Node 65: No energy data, using distance only
  Node 66: No energy data, using distance only
  Node 67: No energy data, using distance only
Chọn Node 66 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 6 nodes ===
  Node 10: No energy data, using distance only
  Node 18: No energy data, using distance only
  Node 19: No energy data, using distance only
  Node 27: No energy data, using distance only
  Node 74: No energy data, using distance only
  Node 82: No energy data, using distance only
Chọn Node 18 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 7 nodes ===
  Node 75: No energy 

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10640\2894817846.py:43: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', len(clusters_output))


Đã lưu hình vẽ D:\Year 4\tiến hóa\project\UWSN_greedy\draw_output_kmeans\nodes_500.png


In [19]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

input_folder = "D:\\Year 4\\tiến hóa\\project\\UWSN_greedy\\input_data_even"
output_folder = "D:\\Year 4\\tiến hóa\\project\\UWSN_greedy\\output_data_kmeans"
os.makedirs(output_folder, exist_ok=True)
draw_folder = "draw_output_kmeans"
os.makedirs(draw_folder, exist_ok=True)

# calculate_number_clusters(), cluster_split(), choose_cluster_head()

def plot_sphere(ax, center, radius, color):
    """Vẽ hình cầu bán trong suốt quanh tâm cụm."""
    u = np.linspace(0, 2 * np.pi, 30)
    v = np.linspace(0, np.pi, 20)
    x = center[0] + radius * np.outer(np.cos(u), np.sin(v))
    y = center[1] + radius * np.outer(np.sin(u), np.sin(v))
    z = center[2] + radius * np.outer(np.ones_like(u), np.cos(v))
    ax.plot_surface(x, y, z, color=color, alpha=0.15, linewidth=0)

for filename in os.listdir(input_folder):
    if filename.startswith("nodes_") and filename.endswith(".json"):
        # Lấy số lượng node từ tên file
        number_nodes = filename.split("_")[1].split(".")[0]
        # Đọc dữ liệu từ file input
        with open(os.path.join(input_folder, filename), "r") as f:
            data = json.load(f)
        node_positions = np.array([[d["x"], d["y"], d["z"]] for d in data])
        node_ids = [d["id"] for d in data]

        # Phân cụm
        k = calculate_number_clusters(node_positions, base_station=(0, 0, 0))
        print(f"Số cụm: {k}")
        clusters_raw = cluster_split(node_positions, node_ids, R=int(number_nodes)//k)

        # Tạo output
        clusters_output = {}
        for i, c in enumerate(clusters_raw):
            ch = choose_cluster_head(c, c["node_data"])
            clusters_output[i] = {
                "nodes": c["node_ids"],
                "center": tuple(np.round(c["center"], 2)),
                "cluster_head": int(ch)
            }

        # Xuất ra file
        out_path = os.path.join(output_folder, f"nodes_{number_nodes}.json")
        with open(out_path, "w") as f:
            json.dump(clusters_output, f, indent=4)
        print(f"Đã xuất file {out_path}")

        # --- Vẽ cụm và vùng truyền thông ---
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')
        colors = plt.cm.get_cmap('tab10', len(clusters_output))

        # Giả sử bán kính truyền thông là R = 20
        communication_range = 50  

        for cid, info in clusters_output.items():
            # Node trong cụm
            nodes = np.array([node_positions[nid] for nid in info['nodes']])
            ax.scatter(nodes[:, 0], nodes[:, 1], nodes[:, 2],
                       label=f'Cụm {cid}', color=colors(cid))

            # Cluster head
            ch_pos = node_positions[info['cluster_head']]
            ax.scatter(ch_pos[0], ch_pos[1], ch_pos[2],
                       color=colors(cid), marker='*', s=100, edgecolor='k')

            # Vẽ vùng phạm vi truyền thông (hình cầu)
            plot_sphere(ax, ch_pos, communication_range, colors(cid))

        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.legend()
        plt.title(f'Phân cụm các node cảm biến ({number_nodes} node)')
        ax.view_init(elev=30, azim=30)
        plt.tight_layout()

        draw_path = os.path.join(draw_folder, f"nodes_{number_nodes}.png")
        plt.savefig(draw_path)
        plt.close(fig)
        print(f"Đã lưu hình vẽ {draw_path}")


N = 100, d_tobs = 360.48, K_optimal = 6
Số cụm: 6

=== Chọn Cluster Head cho cụm có 6 nodes ===
  Node 10: No energy data, using distance only
  Node 15: No energy data, using distance only
  Node 16: No energy data, using distance only
  Node 17: No energy data, using distance only
  Node 20: No energy data, using distance only
  Node 21: No energy data, using distance only
Chọn Node 16 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 2 nodes ===
  Node 45: No energy data, using distance only
  Node 46: No energy data, using distance only
Chọn Node 45 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 8 nodes ===
  Node 35: No energy data, using distance only
  Node 36: No energy data, using distance only
  Node 37: No energy data, using distance only
  Node 40: No energy data, using distance only
  Node 41: No energy data, using distance only
  Node 60: No energy data, using distance only
  Node 61: No energy data, using distance only
  Node 66: No

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10640\2780226495.py:58: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', len(clusters_output))
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10640\2780226495.py:86: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig(draw_path)


Đã lưu hình vẽ draw_output_kmeans\nodes_100.png
N = 1000, d_tobs = 384.50, K_optimal = 18
Số cụm: 18

=== Chọn Cluster Head cho cụm có 16 nodes ===
  Node 0: No energy data, using distance only
  Node 1: No energy data, using distance only
  Node 10: No energy data, using distance only
  Node 11: No energy data, using distance only
  Node 100: No energy data, using distance only
  Node 101: No energy data, using distance only
  Node 102: No energy data, using distance only
  Node 110: No energy data, using distance only
  Node 111: No energy data, using distance only
  Node 112: No energy data, using distance only
  Node 200: No energy data, using distance only
  Node 201: No energy data, using distance only
  Node 202: No energy data, using distance only
  Node 210: No energy data, using distance only
  Node 211: No energy data, using distance only
  Node 212: No energy data, using distance only
Chọn Node 101 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 16 nodes

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10640\2780226495.py:58: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', len(clusters_output))
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10640\2780226495.py:86: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig(draw_path)


Đã lưu hình vẽ draw_output_kmeans\nodes_1000.png
N = 20, d_tobs = 354.69, K_optimal = 3
Số cụm: 3

=== Chọn Cluster Head cho cụm có 2 nodes ===
  Node 10: No energy data, using distance only
  Node 13: No energy data, using distance only
Chọn Node 13 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 1 nodes ===
  Node 0: No energy data, using distance only
Chọn Node 0 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 3 nodes ===
  Node 9: No energy data, using distance only
  Node 18: No energy data, using distance only
  Node 19: No energy data, using distance only
Chọn Node 18 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 2 nodes ===
  Node 2: No energy data, using distance only
  Node 5: No energy data, using distance only
Chọn Node 2 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 1 nodes ===
  Node 1: No energy data, using distance only
Chọn Node 1 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head

...

Đã lưu hình vẽ draw_output_kmeans\nodes_20.png
N = 200, d_tobs = 370.75, K_optimal = 8
Số cụm: 8

=== Chọn Cluster Head cho cụm có 5 nodes ===
  Node 150: No energy data, using distance only
  Node 151: No energy data, using distance only
  Node 156: No energy data, using distance only
  Node 186: No energy data, using distance only
  Node 192: No energy data, using distance only
Chọn Node 156 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 7 nodes ===
  Node 121: No energy data, using distance only
  Node 157: No energy data, using distance only
  Node 162: No energy data, using distance only
  Node 163: No energy data, using distance only
  Node 164: No energy data, using distance only
  Node 198: No energy data, using distance only
  Node 199: No energy data, using distance only
Chọn Node 163 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 8 nodes ===
  Node 116: No energy data, using distance only
  Node 122: No energy data, using distance on

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10640\2780226495.py:58: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', len(clusters_output))
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10640\2780226495.py:86: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig(draw_path)


Đã lưu hình vẽ draw_output_kmeans\nodes_200.png
N = 500, d_tobs = 380.04, K_optimal = 13
Số cụm: 13

=== Chọn Cluster Head cho cụm có 7 nodes ===
  Node 1: No energy data, using distance only
  Node 2: No energy data, using distance only
  Node 3: No energy data, using distance only
  Node 11: No energy data, using distance only
  Node 65: No energy data, using distance only
  Node 66: No energy data, using distance only
  Node 67: No energy data, using distance only
Chọn Node 66 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 6 nodes ===
  Node 10: No energy data, using distance only
  Node 18: No energy data, using distance only
  Node 19: No energy data, using distance only
  Node 27: No energy data, using distance only
  Node 74: No energy data, using distance only
  Node 82: No energy data, using distance only
Chọn Node 18 làm Cluster Head (Q_max = -1.000)

=== Chọn Cluster Head cho cụm có 7 nodes ===
  Node 75: No energy data, using distance only
  Node 146: N

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10640\2780226495.py:58: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('tab10', len(clusters_output))
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10640\2780226495.py:86: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  plt.savefig(draw_path)


Đã lưu hình vẽ draw_output_kmeans\nodes_500.png
